In [1]:
import os
import gc
import math

# numpy
import numpy as np
import numpy.ma as ma

from jax import vmap, grad, jit, random
from jax.config import config
# from jax.experimental import optimizers
import jax.numpy as jnp
from jax.tree_util import tree_map, tree_flatten, tree_unflatten, tree_leaves

config.update("jax_enable_x64", True)

# from skimage.restoration import estimate_sigma
from skimage.transform import resize

### CRPS
import properscoring as ps

### SSIM and PSNR
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import filters

# plotting
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('default')

from matplotlib.gridspec import GridSpec
from matplotlib import colors
from cartopy import config
import cartopy.crs as ccrs

# netCDF
import netCDF4 as nc

from tqdm.notebook import tqdm
from tqdm.notebook import trange

import pickle

In [2]:
xhist_tas = pickle.load(open('../data/saved/xhist_tas.pkl', 'rb'))
xrcp_tas = pickle.load(open('../data/saved/xrcp_tas.pkl', 'rb'))

xhist_pr = pickle.load(open('../data/saved/xhist_pr.pkl', 'rb'))
xrcp_pr = pickle.load(open('../data/saved/xrcp_pr.pkl', 'rb'))

In [3]:
nval = 72
nmod = len(xhist_tas)

In [4]:
sgpr_tas = [np.load(f'tas_predictions/sgpr_tas_{m}.npz.npy') for m in trange(nmod)]
xreg_tas = [np.load(f'tas_predictions/xreg_tas_{m}.npz.npy') for m in trange(nmod)]
xbar_tas = [np.load(f'tas_predictions/xbar_tas_{m}.npz.npy') for m in trange(nmod)]
xwea_tas = [np.load(f'tas_predictions/xwea_tas_{m}.npz.npy') for m in trange(nmod)]
gpse_tas = [np.load(f'tas_predictions/gpr_se_tas_{m}.npz.npy') for m in trange(nmod)]
gpex_tas = [np.load(f'tas_predictions/gpr_exp_tas_{m}.npz.npy') for m in trange(nmod)]
dcnn_tas = [np.load(f'tas_predictions/cnn_tas_{m}.npz.npy') for m in trange(nmod)]
delt_tas = [np.load(f'tas_predictions/del_tas_{m}.npz.npy') for m in trange(nmod)]

sgpr_pr = [np.load(f'pr_predictions/sgpr_pr_{m}.npz.npy') for m in trange(nmod)]
xreg_pr = [np.load(f'pr_predictions/xreg_pr_{m}.npz.npy') for m in trange(nmod)]
xbar_pr = [np.load(f'pr_predictions/xbar_pr_{m}.npz.npy') for m in trange(nmod)]
xwea_pr = [np.load(f'pr_predictions/xwea_pr_{m}.npz.npy') for m in trange(nmod)]
gpse_pr = [np.load(f'pr_predictions/gpr_se_pr_{m}.npz.npy') for m in trange(nmod)]
gpex_pr = [np.load(f'pr_predictions/gpr_exp_pr_{m}.npz.npy') for m in trange(nmod)]
dcnn_pr = [np.load(f'pr_predictions/cnn_pr_{m}.npz.npy') for m in trange(nmod)]
delt_pr = [np.load(f'pr_predictions/del_pr_{m}.npz.npy') for m in trange(nmod)]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [6]:
ntest = sgpr_tas[0].shape[0] + nval

In [7]:
months = 12 * 10
decades = int((ntest-nval)/(months))

In [8]:
sgpr_mse_tas = np.zeros((decades, nmod))
xreg_mse_tas = np.zeros((decades, nmod))
xwea_mse_tas = np.zeros((decades, nmod))
xbar_mse_tas = np.zeros((decades, nmod))
gpse_mse_tas = np.zeros((decades, nmod))
gpex_mse_tas = np.zeros((decades, nmod))
dcnn_mse_tas = np.zeros((decades, nmod))
delt_mse_tas = np.zeros((decades, nmod))

sgpr_mse_pr = np.zeros((decades, nmod))
xreg_mse_pr = np.zeros((decades, nmod))
xwea_mse_pr = np.zeros((decades, nmod))
xbar_mse_pr = np.zeros((decades, nmod))
gpse_mse_pr = np.zeros((decades, nmod))
gpex_mse_pr = np.zeros((decades, nmod))
dcnn_mse_pr = np.zeros((decades, nmod))
delt_mse_pr = np.zeros((decades, nmod))

sgpr_ssim_tas = np.zeros((decades, nmod))
xreg_ssim_tas = np.zeros((decades, nmod))
xwea_ssim_tas = np.zeros((decades, nmod))
xbar_ssim_tas = np.zeros((decades, nmod))
gpse_ssim_tas = np.zeros((decades, nmod))
gpex_ssim_tas = np.zeros((decades, nmod))
dcnn_ssim_tas = np.zeros((decades, nmod))
delt_ssim_tas = np.zeros((decades, nmod))

sgpr_ssim_pr = np.zeros((decades, nmod))
xreg_ssim_pr = np.zeros((decades, nmod))
xwea_ssim_pr = np.zeros((decades, nmod))
xbar_ssim_pr = np.zeros((decades, nmod))
gpse_ssim_pr = np.zeros((decades, nmod))
gpex_ssim_pr = np.zeros((decades, nmod))
dcnn_ssim_pr = np.zeros((decades, nmod))
delt_ssim_pr = np.zeros((decades, nmod))

for m1 in trange(nmod):
    ytest_tas = np.array(xrcp_tas[m1])[nval:ntest]
    ytest_pr = np.array(xrcp_pr[m1])[nval:ntest]
    
    nlat = ytest_tas.shape[1]
    scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))
    
    for d in range(decades):
        
        ytest_dec = ytest_tas[(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        
        sgpr_dec = sgpr_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        xreg_dec = xreg_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        xwea_dec = xwea_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        xbar_dec = xbar_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        gpse_dec = gpse_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        gpex_dec = gpex_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        dcnn_dec = dcnn_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        delt_dec = delt_tas[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        
        sgpr_mse_tas[d, m1] = np.mean((sgpr_dec - ytest_dec)**2)
        xreg_mse_tas[d, m1] = np.mean((xreg_dec - ytest_dec)**2)
        xwea_mse_tas[d, m1] = np.mean((xwea_dec - ytest_dec)**2)
        xbar_mse_tas[d, m1] = np.mean((xbar_dec - ytest_dec)**2)
        gpse_mse_tas[d, m1] = np.mean((gpse_dec - ytest_dec)**2)
        gpex_mse_tas[d, m1] = np.mean((gpex_dec - ytest_dec)**2)
        dcnn_mse_tas[d, m1] = np.mean((dcnn_dec - ytest_dec)**2)
        delt_mse_tas[d, m1] = np.mean((delt_dec - ytest_dec)**2)
        
        sgpr_ssim_tas[d, m1] = ssim(sgpr_dec, ytest_dec)
        xreg_ssim_tas[d, m1] = ssim(xreg_dec, ytest_dec)
        xwea_ssim_tas[d, m1] = ssim(xwea_dec, ytest_dec)
        xbar_ssim_tas[d, m1] = ssim(xbar_dec, ytest_dec)
        gpse_ssim_tas[d, m1] = ssim(gpse_dec, ytest_dec)
        gpex_ssim_tas[d, m1] = ssim(gpex_dec, ytest_dec)
        dcnn_ssim_tas[d, m1] = ssim(dcnn_dec, ytest_dec)
        delt_ssim_tas[d, m1] = ssim(delt_dec, ytest_dec)


        ytest_dec = ytest_pr[(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]

        sgpr_dec = sgpr_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        xreg_dec = xreg_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        xwea_dec = xwea_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        xbar_dec = xbar_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        gpse_dec = gpse_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        gpex_dec = gpex_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        dcnn_dec = dcnn_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        delt_dec = delt_pr[m1][(months*d):(months*(d+1))].astype('float32') * scale[None,:,None]
        
        sgpr_mse_pr[d, m1] = np.mean((sgpr_dec - ytest_dec)**2)
        xreg_mse_pr[d, m1] = np.mean((xreg_dec - ytest_dec)**2)
        xwea_mse_pr[d, m1] = np.mean((xwea_dec - ytest_dec)**2)
        xbar_mse_pr[d, m1] = np.mean((xbar_dec - ytest_dec)**2)
        gpse_mse_pr[d, m1] = np.mean((gpse_dec - ytest_dec)**2)
        gpex_mse_pr[d, m1] = np.mean((gpex_dec - ytest_dec)**2)
        dcnn_mse_pr[d, m1] = np.mean((dcnn_dec - ytest_dec)**2)
        delt_mse_pr[d, m1] = np.mean((delt_dec - ytest_dec)**2)
        
        sgpr_ssim_pr[d, m1] = ssim(sgpr_dec, ytest_dec)
        xreg_ssim_pr[d, m1] = ssim(xreg_dec, ytest_dec)
        xwea_ssim_pr[d, m1] = ssim(xwea_dec, ytest_dec)
        xbar_ssim_pr[d, m1] = ssim(xbar_dec, ytest_dec)
        gpse_ssim_pr[d, m1] = ssim(gpse_dec, ytest_dec)
        gpex_ssim_pr[d, m1] = ssim(gpex_dec, ytest_dec)
        dcnn_ssim_pr[d, m1] = ssim(dcnn_dec, ytest_dec)
        delt_ssim_pr[d, m1] = ssim(delt_dec, ytest_dec)

  0%|          | 0/16 [00:00<?, ?it/s]

In [12]:
from tabulate import tabulate

tas_metrics = np.array([np.arange(1, 9),
          np.mean(sgpr_mse_tas, axis = 1),
          np.mean(xreg_mse_tas, axis = 1),
          np.mean(xwea_mse_tas, axis = 1),
          np.mean(xbar_mse_tas, axis = 1),
          np.mean(gpse_mse_tas, axis = 1),
          np.mean(gpex_mse_tas, axis = 1),
          np.mean(dcnn_mse_tas, axis = 1),
          np.mean(delt_mse_tas, axis = 1),
          np.mean(sgpr_ssim_tas, axis = 1),
          np.mean(xreg_ssim_tas, axis = 1),
          np.mean(xwea_ssim_tas, axis = 1),
          np.mean(xbar_ssim_tas, axis = 1),
          np.mean(gpse_ssim_tas, axis = 1),
          np.mean(gpex_ssim_tas, axis = 1),
          np.mean(dcnn_ssim_tas, axis = 1),
          np.mean(delt_ssim_tas, axis = 1)]).T

pr_metrics = np.array([np.arange(1, 9),
          np.mean(sgpr_mse_pr, axis = 1),
          np.mean(xreg_mse_pr, axis = 1),
          np.mean(xwea_mse_pr, axis = 1),
          np.mean(xbar_mse_pr, axis = 1),
          np.mean(gpse_mse_pr, axis = 1),
          np.mean(gpex_mse_pr, axis = 1),
          np.mean(dcnn_mse_pr, axis = 1),
          np.mean(delt_mse_pr, axis = 1),
          np.mean(sgpr_ssim_pr, axis = 1),
          np.mean(xreg_ssim_pr, axis = 1),
          np.mean(xwea_ssim_pr, axis = 1),
          np.mean(xbar_ssim_pr, axis = 1),
          np.mean(gpse_ssim_pr, axis = 1),
          np.mean(gpex_ssim_pr, axis = 1),
          np.mean(dcnn_ssim_pr, axis = 1),
          np.mean(delt_ssim_pr, axis = 1)]).T

print(tabulate(np.round(tas_metrics, 2),
               numalign = 'center',
               headers = ['NN-GPR', 'REG', 'WEA', 'EA', 'GPSE', 'GPEX', 'CNN', 'DELT'] * 2,
               tablefmt = "latex_booktabs"))

print(tabulate(np.round(pr_metrics, 2),
               numalign = 'center',
               headers = ['NN-GPR', 'REG', 'WEA', 'EA', 'GPSE', 'GPEX', 'CNN', 'DELT'] * 2,
               tablefmt = "latex_booktabs"))

\begin{tabular}{ccccccccccccccccc}
\toprule
    &  NN-GPR  &  REG  &  WEA  &  EA  &  GPSE  &  GPEX  &  CNN  &  DELT  &  NN-GPR  &  REG  &  WEA  &  EA  &  GPSE  &  GPEX  &  CNN  &  DELT  \\
\midrule
 1  &   0.66   &  0.8  & 1.23  & 2.1  &  0.68  &  0.66  & 1.03  &  1.06  &   0.93   & 0.92  & 0.89  & 0.84 &  0.92  &  0.93  &  0.9  &  0.9   \\
 2  &   0.69   &  0.8  & 1.24  & 2.08 &  0.73  &  0.71  & 1.02  &  1.06  &   0.93   & 0.91  & 0.89  & 0.84 &  0.92  &  0.92  &  0.9  &  0.9   \\
 3  &   0.73   & 0.82  & 1.28  & 2.11 &  0.85  &  0.8   & 1.03  &  1.1   &   0.92   & 0.91  & 0.89  & 0.84 &  0.91  &  0.91  &  0.9  &  0.89  \\
 4  &   0.8    & 0.88  & 1.33  & 2.15 &   1    &  0.93  & 1.09  &  1.16  &   0.91   & 0.91  & 0.88  & 0.84 &  0.9   &  0.91  & 0.89  &  0.89  \\
 5  &   0.84   &  0.9  & 1.34  & 2.15 &  1.15  &  1.05  &  1.1  &  1.16  &   0.91   &  0.9  & 0.88  & 0.84 &  0.89  &  0.9   & 0.89  &  0.89  \\
 6  &   0.89   & 0.91  & 1.36  & 2.16 &  1.33  &  1.16  &  1.1  &  1.18  &   